In [8]:
# Import required libraries
from monai.utils import first, set_determinism
from monai.transforms import (
AsDiscrete,
AsDiscreted,
EnsureChannelFirstd,
Compose,
CropForegroundd,
LoadImaged,
Orientationd,
Spacingd,
Invertd,
)
from monai.handlers.utils import from_engine
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
from monai.losses import DiceLoss
from monai.metrics import DiceMetric

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob

from sklearn.model_selection import train_test_split
from skimage.measure import label, regionprops

import SimpleITK as sitk


directory = os.environ.get("MONAI_DATA_DIRECTORY")
root_dir = tempfile.mkdtemp() if directory is None else directory
print(f"root dir is: {root_dir}")

# Define paths to the folders containing images and masks
data_folder = "/projectnb/ec500kb/projects/Project_3/Maolin/Data/"
masks_folder = "/projectnb/ec500kb/projects/Project_3/Maolin/Data/"  # Same as the image folder

# Create empty lists to store image and mask data
image_dataset = []
left_mask_dataset = []
right_mask_dataset = []


# Iterate through image files and import them
for images_folder in os.listdir(data_folder):
    if os.path.isdir(os.path.join(data_folder, images_folder)):
        print(images_folder)
        for filename in os.listdir(os.path.join(data_folder, images_folder)):
            if filename.endswith('.nrrd'):
                # Construct full path to the image file
                raw_img_path = os.path.join(data_folder, images_folder, filename)

                # Read in image
                raw_img_sitk = sitk.ReadImage(raw_img_path, sitk.sitkFloat32)

                # Convert the array
                raw_img_sitk_arr = sitk.GetArrayFromImage(raw_img_sitk)

                # Add image data to list
                image_dataset.append(raw_img_sitk_arr)

                # Iterate through each mask file
                for side in ["left", "right"]:  # It is done this way so that I import 5 copies of each mask. since the same mask is used for each image volume
                    mask_filename = f"svr_{side}KidneyMask2.nii.gz"
                    mask_path = os.path.join(masks_folder, images_folder, mask_filename)

                    # Read in mask
                    raw_mask_sitk = sitk.ReadImage(mask_path, sitk.sitkFloat32)

                    # Convert the array
                    mask_arr = sitk.GetArrayFromImage(raw_mask_sitk)

                    if side == "left":
                        left_mask_dataset.append(mask_arr[0])
                    elif side == "right":
                        right_mask_dataset.append(mask_arr[0])

# Convert lists to numpy arrays
image_dataset = np.array(image_dataset)
left_mask_dataset = np.array(left_mask_dataset)
right_mask_dataset = np.array(right_mask_dataset)

# Print the shapes to verify
print("Image dataset shape:", image_dataset.shape)  # samples, slices, slice_x, slice_y
print("Left Mask dataset shape:", left_mask_dataset.shape)
print("Right Mask dataset shape:", right_mask_dataset.shape)

combined_masks = []

for i in range(len(left_mask_dataset)):
    combined_masks.append(left_mask_dataset[i] + right_mask_dataset[i])

combined_masks = np.array(combined_masks)

root dir is: /scratch/6983143.1.academic-gpu/tmp5zyg7zwb
case7
case3
case4
case5
case8
case1
case2
case6
Image dataset shape: (40, 18, 128, 128)
Left Mask dataset shape: (40, 128, 128)
Right Mask dataset shape: (40, 128, 128)


In [23]:
def bounding_box_edges(mask):
    labeled_mask = label(mask)
    
    bboxes = []
    if labeled_mask.max() == 0:
        return mask
    
    border = np.zeros_like(mask)
    
    for region in regionprops(labeled_mask):
        min_row, min_col, max_row, max_col = region.bbox
        # Fill in bounding box edges
        border[min_row:min_row+2, min_col:max_col+1] = 1  # top edge
        border[max_row-1:max_row+1, min_col:max_col+1] = 1  # bottom edge
        border[min_row:max_row+1, min_col:min_col+2] = 1  # left edge
        border[min_row:max_row+1, max_col-1:max_col+1] = 1  # right edge
    
    return border

def calculate_iou(bbox1, bbox2):
    # Calculate intersection
    intersection = np.logical_and(bbox1, bbox2)

    # Calculate union
    union = np.logical_or(bbox1, bbox2)

    # Calculate IoU
    iou = np.sum(intersection) / np.sum(union)

    return iou

In [12]:
np.shape(combined_masks)

(40, 128, 128)

In [10]:
"""
Define the volume U-Net Model
"""
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define the UNet model
class MyUNet(nn.Module):
    def __init__(self):
        super(MyUNet, self).__init__()
        self.unet = UNet(
            spatial_dims=2,  # Set spatial dimensions to 2 for 2D data
            in_channels=18,  # Input volume with 18 channels
            out_channels=1,  # Single channel output mask
            channels=(16, 32, 64, 128, 256), #, 256 # Depth of 5
            strides=(2, 2, 2, 2),
            num_res_units=2,  # Number of residual units
            dropout=0.1
        )

    def forward(self, x):
        return self.unet(x)


Using device: cuda


In [33]:
"""
Loop to do every possible 7/1 split in the participant population
"""
MEAN_TEST_LOSS = []
MEAN_DICE = []
MEAN_IOU = []

# Use each participant as the test once
for test_idx in range(8):
    print(test_idx)
    big_loss = []

    # Define a new model for each run through
    model = MyUNet().to(device)

    # Define Dice Loss
    criterion = DiceLoss(sigmoid=True)

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Create the training test split
    test_start = test_idx * 5
    test_end = (test_idx + 1) * 5

    # Define training and testing data
    X_train = np.concatenate([image_dataset[:test_start], image_dataset[test_end:]])
    y_train = np.concatenate([combined_masks[:test_start], combined_masks[test_end:]])
    X_test = image_dataset[test_start:test_end]
    y_test = combined_masks[test_start:test_end]
    
    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)  # add channel dimension
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)  # add channel dimension

    # Train the model on this training set
    num_epochs = 1000
    for epoch in range(num_epochs):
        # Set the model to train mode
        model.train()

        # Forward pass
        outputs = model(X_train_tensor)

        # Compute the loss
        loss = criterion(outputs, y_train_tensor)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        big_loss.append(loss.item())
        # Print loss every 10 epochs
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

    # After training, predict on the test set
    model.eval()
    with torch.no_grad():
        outputs = model(X_test_tensor)
        test_loss = criterion(outputs, y_test_tensor)
        print(f'Test Loss: {test_loss.item():.4f}')
        test_masks = [thing.cpu() for thing in outputs]

    # Binarize the masks
    thresholded_masks = []

    for mask in test_masks:
        # Calculate intensity range
        min_intensity = torch.min(mask)
        max_intensity = torch.max(mask)
        intensity_range = max_intensity - min_intensity

        # Calculate 50% mark
        threshold = min_intensity + 0.5 * intensity_range

        # Threshold the mask
        thresholded_mask = torch.where(mask >= threshold, torch.tensor(1), torch.tensor(0))

        # Append the thresholded mask to the list
        thresholded_masks.append(thresholded_mask)
    
    dice_metric = DiceMetric(include_background=True)

    # Calculate Dice score
    test_dice = []

    for idx in range(len(test_masks)):
        # Get proper shapes for dice score function
        y_true = torch.tensor(y_test[idx]).unsqueeze(0).unsqueeze(0)
        y_pred = torch.tensor(thresholded_masks[idx]).unsqueeze(0).unsqueeze(0)

        # Calculate the dice score :P
        dice = dice_metric(y_pred, y_true)

        # append to the list of dice
        test_dice.append(float(dice))

    # Calculate mean dice
    MEAN_DICE.append(np.mean(test_dice))

    
    # Calculate the IOU
    # Create bounding box coordinates
    mask_array = []
    bboxes = []
    for i in range(len(y_test)):
        mask_array.append(thresholded_masks[i][0].numpy())
        bboxes.append(bounding_box_edges(mask_array[i]))

    IOU_storage = []
    for i in range(len(mask_array)):
        IOU_storage.append(calculate_iou(mask_array[i], y_test[i]))

    # Calculate mean IOU
    MEAN_IOU.append(np.mean(IOU_storage))

    # Calculate test loss
    test_loss = criterion(outputs, y_test_tensor)
    MEAN_TEST_LOSS.append(test_loss.item())

0
Epoch [100/1000], Loss: 0.5337
Epoch [200/1000], Loss: 0.3131
Epoch [300/1000], Loss: 0.1928
Epoch [400/1000], Loss: 0.1118
Epoch [500/1000], Loss: 0.0669
Epoch [600/1000], Loss: 0.0434
Epoch [700/1000], Loss: 0.0301
Epoch [800/1000], Loss: 0.0223
Epoch [900/1000], Loss: 0.0170
Epoch [1000/1000], Loss: 0.0135
Test Loss: 0.3164
1


/scratch/6983143.1.academic-gpu/ipykernel_2310558/1111044313.py:94: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(thresholded_masks[idx]).unsqueeze(0).unsqueeze(0)


Epoch [100/1000], Loss: 0.6012
Epoch [200/1000], Loss: 0.3047
Epoch [300/1000], Loss: 0.1716
Epoch [400/1000], Loss: 0.1153
Epoch [500/1000], Loss: 0.0837
Epoch [600/1000], Loss: 0.0632
Epoch [700/1000], Loss: 0.0497
Epoch [800/1000], Loss: 0.0406
Epoch [900/1000], Loss: 0.0339
Epoch [1000/1000], Loss: 0.0287
Test Loss: 0.3433
2
Epoch [100/1000], Loss: 0.5770
Epoch [200/1000], Loss: 0.3572
Epoch [300/1000], Loss: 0.2354
Epoch [400/1000], Loss: 0.1453
Epoch [500/1000], Loss: 0.0884
Epoch [600/1000], Loss: 0.0569
Epoch [700/1000], Loss: 0.0392
Epoch [800/1000], Loss: 0.0285
Epoch [900/1000], Loss: 0.0216
Epoch [1000/1000], Loss: 0.0170
Test Loss: 0.2033
3
Epoch [100/1000], Loss: 0.5205
Epoch [200/1000], Loss: 0.2950
Epoch [300/1000], Loss: 0.1786
Epoch [400/1000], Loss: 0.1052
Epoch [500/1000], Loss: 0.0651
Epoch [600/1000], Loss: 0.0430
Epoch [700/1000], Loss: 0.0302
Epoch [800/1000], Loss: 0.0224
Epoch [900/1000], Loss: 0.0173
Epoch [1000/1000], Loss: 0.0138
Test Loss: 0.2690
4
Epoch [

In [36]:
#print(MEAN_TEST_LOSS)
#print(MEAN_DICE)
print(MEAN_IOU)

[0.4628210102221385, 0.49725355650549624, 0.5913396760982835, 0.5862151167271445, 0.7374472674422631, 0.5552064916088438, 0.7635396674521673, 0.7405907226569177]
